In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [27]:
# The code was removed by Watson Studio for sharing.

In [28]:
torono_df = torono_df.dropna()
torono_df.head()

,Postcode,borough,neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [29]:
geolocator = Nominatim(user_agent='myapp')
location = geolocator.geocode('Toronto, CA')
latitude = location.latitude
longitude = location.longitude

In [30]:

toronto_map = folium.Map(location=[latitude,longitude],zoom_start=13)
toronto_map

In [31]:
for lat,lon, borough, neighbourhood in zip(torono_df['latitude'], torono_df['longitude'], torono_df['borough'], torono_df['neighbourhood']):
    #print("Borough - {}, Neighbourhood - {}, latitude - {}, longitude - {}".format(borough,neighbourhood,lat,lon))
    label = folium.Popup(neighbourhood,parse_html=True)
    
    folium.CircleMarker(
        location=[lat,lon],
        color = 'red',
        radius = 5,
        popup=label,
        parse_html = True).add_to(toronto_map)
display(toronto_map)

In [101]:
# The code was removed by Watson Studio for sharing.

In [102]:
#for lat,lon, neighbourhood, Postcode in zip(torono_df['latitude'],torono_df['longitude'],torono_df['neighbourhood'],torono_df['borough']):

radius = 700
latitude  = torono_df.loc[torono_df['neighbourhood'] == 'Woburn']['latitude'].values[0]
longitude = torono_df.loc[torono_df['neighbourhood'] == 'Woburn']['longitude'].values[0]
search_query = 'Woburn'
LIMIT = 100
print(latitude)
print(longitude)
print(search_query)

43.7712
-79.2144
Woburn


In [103]:
# for each in search_query:
#     url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, each, radius, LIMIT)
#     url
#     results = requests.get(url).json()
#     print(results)
       
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
results = requests.get(url).json()
print(results)


{'meta': {'code': 200, 'requestId': '5e5ce04ddd0f850028479718'}, 'response': {'headerLocation': 'Toronto', 'headerFullLocation': 'Toronto', 'headerLocationGranularity': 'city', 'totalResults': 6, 'suggestedBounds': {'ne': {'lat': 43.77750000630001, 'lng': -79.20569182467621}, 'sw': {'lat': 43.76489999369999, 'lng': -79.22310817532379}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4cc1d28c06c254815ac18547', 'name': 'Starbucks', 'location': {'address': '300 Borough Dr', 'crossStreet': 'Scarborough Town Centre', 'lat': 43.770037201625215, 'lng': -79.22115586641958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.770037201625215, 'lng': -79.22115586641958}], 'distance': 558, 'cc': 'CA', 'city': 'Scarborough', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['300 Borough Dr (Scarborough Town Centre)', 